In [4]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


from pyntcloud import PyntCloud

from dataset import ShapeNetDataset
from model import ClassificationNN, SegmentationNN, FeatureTransform, train, plot_stats
from torch.utils.data import DataLoader, random_split

In [5]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


![plot](images/model.png)

In [8]:
import subprocess
import os
if os.path.exists('./datasets'):
    pass
else:
    if not os.path.exists('./ModelNet10.zip'):
        subprocess.run(["wget", "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip"])
    subprocess.run(["unzip", "-o", "ModelNet10.zip", "-d", "./datasets/"])



Archive:  ModelNet10.zip
   creating: ./datasets/ModelNet10/
  inflating: ./datasets/ModelNet10/.DS_Store  
   creating: ./datasets/__MACOSX/
   creating: ./datasets/__MACOSX/ModelNet10/
  inflating: ./datasets/__MACOSX/ModelNet10/._.DS_Store  
   creating: ./datasets/ModelNet10/bathtub/
   creating: ./datasets/ModelNet10/bathtub/test/
  inflating: ./datasets/ModelNet10/bathtub/test/bathtub_0107.off  
   creating: ./datasets/__MACOSX/ModelNet10/bathtub/
   creating: ./datasets/__MACOSX/ModelNet10/bathtub/test/
  inflating: ./datasets/__MACOSX/ModelNet10/bathtub/test/._bathtub_0107.off  
  inflating: ./datasets/ModelNet10/bathtub/test/bathtub_0108.off  
  inflating: ./datasets/__MACOSX/ModelNet10/bathtub/test/._bathtub_0108.off  
  inflating: ./datasets/ModelNet10/bathtub/test/bathtub_0109.off  
  inflating: ./datasets/__MACOSX/ModelNet10/bathtub/test/._bathtub_0109.off  
  inflating: ./datasets/ModelNet10/bathtub/test/bathtub_0110.off  
  inflating: ./datasets/__MACOSX/ModelNet10/batht

In [4]:
n = 10000

trainset = ShapeNetDataset("datasets/ModelNet10", train=True, n=n)
testset = ShapeNetDataset("datasets/ModelNet10", train=False, n=n)

dataset_length = len(trainset)
train_ratio = 0.7
test_ratio = 0.3

train_len = int(dataset_length * train_ratio)
test_len = dataset_length - train_len

trainset, testset = random_split(trainset, [train_len, test_len])


In [5]:
num_classes = 10
classification_model = ClassificationNN(num_classes).to(device)

In [6]:
lr = 0.0001
epochs = 1

optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr)
train(classification_model, trainset, testset, optimizer, epochs=epochs, device=device)

  0%|          | 0/88 [00:00<?, ?batch/s]

D:\Users\ALIENWARE\Anaconda3\envs\torch190\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1260.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: CUDA out of memory. Tried to allocate 158.00 MiB (GPU 0; 6.00 GiB total capacity; 5.21 GiB already allocated; 0 bytes free; 5.26 GiB reserved in total by PyTorch)

# ModelNet10 Dataset

The ModelNet10 Dataset features 5586 different pointcloud objects across 12 different classes. Below, we show the train/test split for each of the classes.

In [ ]:
train_indices = trainset.indices
test_indices = testset.indices

dataset = ShapeNetDataset("datasets/ModelNet10", train=True, n=n)

train_class_counts = dataset.df.loc[train_indices, 'label'].value_counts()
test_class_counts = dataset.df.loc[test_indices, 'label'].value_counts()

x_labels = [dataset.classes[label] for label in train_class_counts.index]
x = range(len(x_labels))

plt.bar(x, train_class_counts, label='train', width=0.4)
plt.bar([xi + 0.4 for xi in range(len(x_labels))], test_class_counts, label='test', width=0.4)
plt.xticks([xi for xi in x], x_labels, rotation=45)
plt.ylabel('Number of Samples')
plt.title('Train/Test Dataset Contents')
plt.legend()

Here, we will show an example of a desk pointcloud. We first print out the first point of its representation. Note that it is a set of unordered (X, Y, Z) point.

In [ ]:
for key, value in dataset.classes.items():
    if value == "desk":
        label = key
        break

idx = dataset.df[dataset.df['label'] == label].index[0]
point_cloud = dataset[idx]
first_point = point_cloud[0].tolist()  
print(f"Class: desk")
print("First point xyz coords")
print(f"x: {first_point[0][0]}, y: {first_point[1][0]}, z: {first_point[2][0]}")
print()

Now, we will produce a scatter plot showing the desk's representation by plotting the (x, y, z) points on a 3D scatter plot.

In [ ]:
np_ptcld = point_cloud[0].numpy()
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')
ax.scatter(np_ptcld[0], np_ptcld[1], np_ptcld[2], s=2)
plt.title('Desk Pointcloud')
plt.show()